In [225]:
import re 
import string
import timestring
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [228]:
def tokenize(text):
    punc_re = '[' + '\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\_\\{\\|\\}' + ']'
    #print punc_re
    text = text.lower()
    text = re.sub('#(\S+)', r'HASHTAG_\1', text)
    text = re.sub('@(\S+)', r'MENTION_\1', text)
    text = re.sub('http\S+', 'THIS_IS_A_URL', text)
    text = re.sub(r'(.)\1\1\1+', r'\1', text)
    text = re.sub(r'[0-9]', '9', text)
    toks = []
    for tok in text.split():
        tok = re.sub(r'^(' + punc_re + '+)', r'\1 ', tok)
        #print '1',tok
        tok = re.sub(r'(' + punc_re + '+)$', r' \1', tok)
        #print '2',tok

        for subtok in tok.split():
            #print '3',subtok
            if re.search('\w', subtok):
                toks.append(subtok)
    return toks

In [205]:
DIR = 'D:/TCC/Data/Timeline/vemprarua/'
import os, io, json, codecs

os.chdir(DIR)
date_ative = []
countuser = 0
user = []

#for file in glob.glob("*.txt"):
 #   print 'file ', file, '\n'
with io.open('CleuzaBapti.txt','r',encoding="utf-8") as f, codecs.open("CleuzaBapti - Copy.txt", 'w+', encoding='utf-8') as fi:
    for line in  reversed(f.readlines()):  
        tweet= json.loads(line)
        tweet['tokened_text'] = tokenize(tweet['text'])
        fi.write(json.dumps(tweet,ensure_ascii=False,encoding='utf-8')+'\n')

        #if countuser == 100:
         #   break
        countuser+=1


In [229]:
DIR= 'C:/Users/Elaine/Documents/Data/Timeline/vemprarua/'
os.chdir(DIR)

#for file in glob.glob("*.txt"):
    #print 'file ', file, '\n'
positive_tweets = []
easy_negative_sample = []
hard_negative_sample = []
N=5

with io.open(DIR+'TatianaRCosta.txt.txt','r',encoding="utf-8") as f:
    lines = f.readlines()
    for i in xrange(len(lines)):
        tweet= json.loads(lines[i])
        if 'vempraRua' in tweet['text'] or 'vemprarua' in tweet['text'] or 'vem pra rua' in tweet['text']or 'VEMPRARUA' in tweet['text'] or 'VemPraRua' in tweet['text']:
            positive_tweets.append(tweet)
            date = timestring.Date(tweet['time_tweet'])
            i=i-1
            j=i-1
            tweet_date = date
            while  tweet_date >= date - '7 day': 
                tweet= json.loads(lines[i])                
                if timestring.Date(tweet['time_tweet']) < date - '7 day':
                    print 'Tweet created before 7 days: \n tweet date',timestring.Date(tweet['time_tweet']) ,'<', date - '7 day'
                    break
                else:
                    easy_negative_sample.append(tweet)
                    i=i-1
                    tweet_date = timestring.Date(tweet['time_tweet'])
                if i == 1:
                    print 'comeco do arquivo'
                    break
                    
            while len(hard_negative_sample)<N:
                tweet= json.loads(lines[j])    
                #print tweet['time_tweet']
                hard_negative_sample.append(tweet)
                j=j-1
                if j == 1:
                    print 'comeco do arquivo'
                    break
            break

Tweet created before 7 days: 
 tweet date 2015-08-08 00:00:00 < 2015-08-09 00:00:00


In [230]:
print len(positive_tweets), positive_tweets[0]['created_at'],positive_tweets[0]['text']
print len(easy_negative_sample)

#for f in easy_negative_sample:
 #   print f['time_tweet']
    
print len(hard_negative_sample)
#for f in hard_negative_sample:
 #   print f['time_tweet']

1 Sun Aug 16 19:19:25 +0000 2015 #vemprarua #forapt #londrinanaluta hellen_ukstin @ Praça Da Bandeira https://t.co/CI1yduR61x
7
5


In [231]:
all_tweets = easy_negative_sample+positive_tweets+hard_negative_sample
print len(all_tweets)

13


In [232]:
vectorizer = TfidfVectorizer(decode_error='ignore', ngram_range=(1, 2), max_df=1., min_df=2,
                             use_idf=True, tokenizer=tokenize, binary=False, norm='l2')
X = vectorizer.fit_transform(t['text'] for t in all_tweets)
print('Vectorized %d tweets. Found %d terms.' % (X.shape[0], X.shape[1]))
features = np.array(vectorizer.get_feature_names())

UnicodeEncodeError: 'ascii' codec can't encode character u'\xed' in position 25: ordinal not in range(128)

In [227]:
print features

[u'HASHTAG_bomdia' u'HASHTAG_bomdia HASHTAG_deusnocomando'
 u'HASHTAG_criancaesperanca' u'HASHTAG_deusnocomando'
 u'HASHTAG_deusnocomando HASHTAG_meuanjomeguarda'
 u'HASHTAG_deusnocomando HASHTAG_vivendoeaprendendo'
 u'HASHTAG_for\xe7anaperuca'
 u'HASHTAG_for\xe7anaperuca HASHTAG_vamoqvamo\u2026'
 u'HASHTAG_mariapassanaftrente'
 u'HASHTAG_mariapassanaftrente HASHTAG_for\xe7anaperuca'
 u'HASHTAG_meuanjomeguarda'
 u'HASHTAG_meuanjomeguarda HASHTAG_mariapassanaftrente'
 u'HASHTAG_meuanjomeguarda\u2026'
 u'HASHTAG_meuanjomeguarda\u2026 THIS_IS_A_URL' u'HASHTAG_s\xf3acho'
 u'HASHTAG_s\xf3acho HASHTAG_criancaesperanca' u'HASHTAG_vamoqvamo\u2026'
 u'HASHTAG_vamoqvamo\u2026 THIS_IS_A_URL' u'HASHTAG_vivendoeaprendendo'
 u'HASHTAG_vivendoeaprendendo HASHTAG_meuanjomeguarda\u2026'
 u'MENTION_mjournalstudio' u'MENTION_mjournalstudio THIS_IS_A_URL'
 u'THIS_IS_A_URL' u'THIS_IS_A_URL quer' u'a' u'a brilhar' u'a cantar'
 u'alegria' u'alegria bom' u'amor' u'amor o' u'artista' u'artista fabio'
 u'as' u'